In [1]:
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import tqdm
from qdrant_client import QdrantClient, models

In [3]:
client = OpenAI()

response = client.embeddings.create(
    input="hello",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.01679842546582222, -0.05577957630157471, 0.005663050338625908, 0.06613738089799881, 0.008942044340074062, -0.04729967191815376, -0.02813919633626938, 0.06109052151441574, -0.0022061688359826803, -0.043749261647462845, 0.009858989156782627, -0.03330342844128609, -0.0122063672170043, -0.02887275256216526, 0.01460509467869997, 0.05824432522058487, -0.07006923854351044, 0.03752870857715607, 0.010093726217746735, 0.04078569635748863, 0.06643080711364746, 0.010130404494702816, -0.0055603524670004845, 0.023679178208112717, 0.02174259163439274, 0.011883602477610111, -0.004797454457730055, 0.01930718682706356, 0.024295365437865257, -0.0649636909365654, 0.03330342844128609, -0.044541504234075546, 0.020348835736513138, -0.004999182187020779, 0.010856624692678452, -0.014832496643066406, -0.02626129426062107, 0.034682516008615494, 0.0023253716062754393, -0.03330342844128609, -0.022124040871858597, -0.00856793113052845, 0.05401904135942459, 0.024354049935936928, -0.00011158070265082642, 0.0156247

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
 
client_qd = QdrantClient("127.0.0.1", port=6333)
# client_qd.recreate_collection(
#     collection_name='putin',
#     vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
# )

In [5]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    text = text.replace(" ", "")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

start_index = 30  # 设定起始行数

with open('knowledge.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    total_lines = len(lines)
    for index, line in enumerate(tqdm.tqdm(lines[start_index:], total=total_lines-start_index)):
        embedding = get_embedding(line)
        client_qd.upsert(
            collection_name='putin',
            wait=True,
            points=[
                models.PointStruct(id=start_index + index + 1, vector=embedding, payload={"text": line}),
            ],
        )


100%|██████████| 539/539 [03:47<00:00,  2.37it/s]


In [8]:
question = '中国医学的起源？'
search_results = client_qd.search(
    collection_name='putin',
    query_vector=get_embedding(question),
    limit=10,
    search_params={"exact": False, "hnsw_ef": 128}
)
print(search_results)

[ScoredPoint(id=494, version=2753, score=0.87424946, payload={'text': '01中国医学的起源\n'}, vector=None, shard_key=None), ScoredPoint(id=498, version=2757, score=0.68447423, payload={'text': '一、中国医学的起源◇ 成书于公元前2世纪（春秋战国时期）的《黄帝内经》，奠定了2000余年来中国医药学形成和发展的理论基础，是我国周边国家，如日本汉方医学，朝鲜医、越南医等传统医药学的源头和始祖。◇ 书中记载的临床医学、预防医学体系的理论，以及以针灸为代表的治疗方法仍然沿用至今，对传统医药和未来世界医学多元化的发展产生巨大影响。◇ 2011年3月，我国《黄帝内经》和《本草纲目》入选联合国教科文组织《世界记忆名录》。6.中国医药学文献记载\n'}, vector=None, shard_key=None), ScoredPoint(id=500, version=2759, score=0.62562186, payload={'text': '一、中国医学的起源②.扁鹊         公元前5世纪，河北任丘人。         创脉诊：《史记》：“至今天下言脉者，由扁鹊也。”——诊赵简子（血脉治）、虢太子（尸厥症）——用脉搏、心跳判断死亡沿续了2500年，1980年代才有脑死亡的概念——更早有《周礼》、《脉法》《阴阳病死候》7.古代医家画像石——扁鹊行医（针刺）图（微山县双城山）\n'}, vector=None, shard_key=None), ScoredPoint(id=495, version=2754, score=0.61631984, payload={'text': '一、中国医学的起源1.灸法起源——火的应用\n'}, vector=None, shard_key=None), ScoredPoint(id=497, version=2756, score=0.6159987, payload={'text': '一、中国医学的起源◇ 甲骨文疾首、疾目、疾耳、疾腹，疟、疥、蛊等20多种病名。◇ 《周礼·天官》    四类医生：食医、疾医、疡医、兽医。    五药：草、木、虫、石、

In [10]:
def format_prompt(question, search_results):
    results = ""
    for i in range(10):
        results = results + search_results[i].payload['text']
    # print(results)
    return f'你现在是一个问答知识库，下面是一个题和其难易程度及其答案解析，请你自主分析哪个是答案，并给出回答和解释{results}, 用户给你的询问是{question}'

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": format_prompt(question, search_results)},
  ]
)
print(response.choices[0].message.content)

中国医学的起源可以追溯到公元前2世纪的《黄帝内经》，这部经典奠定了中国医药学的理论基础，影响深远。此外，如扁鹊、淳于意、涪翁、华佗、张仲景等古代医家也在医学史上留下了重要的贡献和成就。因此，中国医学的起源可以说是源远流长，承载着丰富的中华文化和医学智慧。
